## Clasificador de Noticias Argentinas 

In [2]:
import pandas as pd

df = pd.read_excel('../../res/tp1/Noticias_argentinas.xlsx', sheet_name='aa_bayes')

In [3]:
# Nos quedamos con los atributos de interes

df = df[["titular", "categoria"]]

# removemos las categorias que no necesitamos ('Destacadas', 'Noticias Destacadas')


df.head()

,titular,categoria
0,Trabajadores del Buenos Aires Design cortan la...,Nacional
1,La boda del gobernador Gerardo Morales: tapas ...,Nacional
2,Cumbre del G20: qué calles estarán cortadas y ...,Nacional
3,Una fractura que confirma la candidatura de Cr...,Nacional
4,Infierno grande: ola de divorcios en un pueblo...,Nacional


In [4]:
df = df.loc[df['categoria'].isin(("Nacional", "Economia", "Internacional", "Deportes", "Salud", "Ciencia y Tecnologia", "Entretenimiento"))]
df.head()
categories = df['categoria'].unique()
print(categories)


['Nacional' 'Deportes' 'Salud' 'Ciencia y Tecnologia' 'Entretenimiento'
 'Economia' 'Internacional']


In [5]:
len(df)

26961

In [6]:
df['categoria'].value_counts()

Nacional                3860
Ciencia y Tecnologia    3856
Deportes                3855
Economia                3850
Internacional           3850
Entretenimiento         3850
Salud                   3840
Name: categoria, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

# Separar los datos en datos de entrenamiento y testeo
train, test = train_test_split(df, test_size=0.2)
train.head()
test.head()

,titular,categoria
6336,Descuartizó en tres partes a su mujer y escond...,Nacional
2463,"Cómo es Orión, la nave espacial con la que la ...",Ciencia y Tecnologia
10133,Una de cada tres personas con VIH en Latinoamé...,Salud
7123,Trump redobla su amenaza de cerrar la frontera...,Internacional
15372,"ANSES | Adelantan el pago de jubilaciones, pen...",Nacional


In [9]:
textos = list(df['titular'])

In [10]:
# Crear diccionario de categorias con diccionarios vacios para las palabras de los titulares
categories_word_appearances = { i : {} for i in categories }

In [11]:
print(categories_word_appearances)

{'Nacional': {}, 'Deportes': {}, 'Salud': {}, 'Ciencia y Tecnologia': {}, 'Entretenimiento': {}, 'Economia': {}, 'Internacional': {}}


In [12]:
import re

def split_and_sanitize(title):
    words = title.split()  # se separan las palabras según espacios (quita todos los espacios)
    words = list(map(lambda x: re.sub("[^\w\s]", '', x), words))  # reemplazo de símbolos por ningún caracter
    return words

for i in range(len(train)):
    row = train.iloc[i]
    category = row['categoria']
    title = row['titular']
    words = split_and_sanitize(title)
    
    for word in words:
            if word in categories_word_appearances[category]:
                categories_word_appearances[category][word] += 1
            else:
                categories_word_appearances[category][word] = 1
                
    
    
    

In [13]:
print(categories_word_appearances['Entretenimiento']['Pampita'])

68


In [14]:
# Calcular probabilidades de las clases P(category) para el conjunto de entrenamiento

category_relative_freq = train['categoria'].value_counts() / len(train)  # P(category)
print(category_relative_freq)

# Calcular las probabilidades condicionales P(word | category)
word_relative_frequencies = { i : {} for i in categories }
zero_probabilities = { i : 0 for i in categories } # Laplace correction in case a word does not appear in a category
for category in categories:
    word_appearances_sum = sum(categories_word_appearances[category].values())
    laplace_denominator = float(word_appearances_sum + len(categories_word_appearances[category].keys()))
    zero_probabilities[category] = 1 / laplace_denominator
    for word, word_count in categories_word_appearances[category].items():
        word_relative_frequencies[category][word] = (word_count + 1) / laplace_denominator

print(word_relative_frequencies['Entretenimiento']['Pampita'])
# Objetivo final:
#   P (category | [word])
# = P ([word] | category) * P (category)      ##  / P ([word])
# = prod(P (word | category)) * P (category)

Entretenimiento         0.143871
Deportes                0.143500
Ciencia y Tecnologia    0.143175
Nacional                0.142851
Salud                   0.142711
Internacional           0.142387
Economia                0.141506
Name: categoria, dtype: float64
0.0014641599117260111


In [15]:
def classify(title):
    words = split_and_sanitize(title)
    vs = {}
    for category in categories:
        v = category_relative_freq.loc[category]
        for word in words:
            if word in word_relative_frequencies[category]:
                v *= word_relative_frequencies[category][word]
            else:
                v *= zero_probabilities[category]
        vs[category] = v
    mx = (None, 0)
    for category, v in vs.items():
        if v > mx[1]:
            mx = (category, v)
    return mx[0]

noticia = 'Pampita fue a la mesa de Mirtha'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")
noticia = 'Messi metió un gol'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")
noticia = 'Apple lanzó un nuevo iPhone'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")

La noticia 'Pampita fue a la mesa de Mirtha' pertenece a Entretenimiento
La noticia 'Messi metió un gol' pertenece a Deportes
La noticia 'Apple lanzó un nuevo iPhone' pertenece a Ciencia y Tecnologia


In [57]:
confusion_matrix = { i : {} for i in categories }
columns = []
for key in confusion_matrix.keys():
    columns.append(key)
    confusion_matrix[key] = { i : 0 for i in categories } # {'Deportes': {'Deportes': 700, 'Entretenimiento': 3, ...}, ...}

for i in range(len(test)):
    row = test.iloc[i]
    category = row['categoria']
    title = row['titular']
    result = classify(title)
    confusion_matrix[category][result] += 1

print(confusion_matrix)
confusion_df = { key : list(confusion_matrix[key].values()) for key in confusion_matrix.keys() }
print('')
print(confusion_df)
confusion_df = pd.DataFrame.from_dict(confusion_df, orient='index', columns=columns)
print('')
print(confusion_df)

{'Nacional': {'Nacional': 711, 'Deportes': 10, 'Salud': 5, 'Ciencia y Tecnologia': 0, 'Entretenimiento': 7, 'Economia': 20, 'Internacional': 26}, 'Deportes': {'Nacional': 7, 'Deportes': 731, 'Salud': 5, 'Ciencia y Tecnologia': 0, 'Entretenimiento': 11, 'Economia': 2, 'Internacional': 4}, 'Salud': {'Nacional': 1, 'Deportes': 0, 'Salud': 756, 'Ciencia y Tecnologia': 1, 'Entretenimiento': 1, 'Economia': 0, 'Internacional': 3}, 'Ciencia y Tecnologia': {'Nacional': 1, 'Deportes': 0, 'Salud': 5, 'Ciencia y Tecnologia': 747, 'Entretenimiento': 4, 'Economia': 10, 'Internacional': 1}, 'Entretenimiento': {'Nacional': 8, 'Deportes': 4, 'Salud': 6, 'Ciencia y Tecnologia': 6, 'Entretenimiento': 710, 'Economia': 5, 'Internacional': 8}, 'Economia': {'Nacional': 4, 'Deportes': 5, 'Salud': 6, 'Ciencia y Tecnologia': 10, 'Entretenimiento': 1, 'Economia': 765, 'Internacional': 7}, 'Internacional': {'Nacional': 25, 'Deportes': 5, 'Salud': 16, 'Ciencia y Tecnologia': 6, 'Entretenimiento': 10, 'Economia': 1

In [71]:
true = []
predicted = []

for i in range(len(test)):
    row = test.iloc[i]
    true.append(row['categoria'])
    title = row['titular']
    predicted.append(classify(title))

    
# Calculamos las metricas para cada categoria    
    
category_metrics = { i : {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0, 'accuracy': 0, 'precision': 0, 'recall': 0,  'f1': 0} for i in categories }

for i, category in enumerate(categories):
    
    # metricas simples
    tp = confusion_df.iloc[i,i]
    tn = sum(np.diag(confusion_df)) - tp
    fn = sum(confusion_df.loc[category]) - tp
    fp = sum(confusion_df[category]) - tp
    
    category_metrics[category]['tp'] = tp
    category_metrics[category]['tn'] = tn
    category_metrics[category]['fp'] = fp
    category_metrics[category]['fn'] = fn
    
    # metricas compuestas
    category_metrics[category]['accuracy'] = (tp + tn) / (tp + tn + fn + fp)
    category_metrics[category]['precision'] = tp / (tp + fp)
    category_metrics[category]['recall'] = tp / (tp + fn)
    category_metrics[category]['f1'] = (2 * precision * recall) / (precision + recall)

metric_cols = ['tp', 'tn', 'fp', 'fn', 'accuracy', 'precision', 'recall', 'f1']
metrics_df = { key : list(category_metrics[key].values()) for key in category_metrics.keys() }    
metrics_df = pd.DataFrame.from_dict(category_metrics, orient='index', columns=metric_cols)
metrics_df.head()

# Calculamos las metricas para el classifier en general (checkear cual es la forma correcta)
    
tp, fp, fn = 0, 0, 0


for i, category in enumerate(categories):
    tp += confusion_df.iloc[i, i]
    fp += sum(confusion_df.iloc[i, :i])
    fn += sum(confusion_df.iloc[i, i+1:])
    
tn = confusion_df.to_numpy().sum() - (fp + fn + tp)

accuracy = (tp + tn) / (tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = (2 * precision * recall) / (precision + recall)

metrics_df.head()

    



,tp,tn,fp,fn,accuracy,precision,recall,f1
Nacional,711,4413,46,68,0.978236,0.939234,0.912709,0.974422
Deportes,731,4393,24,29,0.989762,0.968212,0.961842,0.974422
Salud,756,4368,43,6,0.990528,0.946183,0.992126,0.974422
Ciencia y Tecnologia,747,4377,23,21,0.991486,0.970130,0.972656,0.974422
Entretenimiento,710,4414,34,37,0.986333,0.954301,0.950469,0.974422


In [ ]:
# Calcular curva ROC

u = [0.1, 0.2, 0.3, 0.4, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

